### Can we classify each phase as the TSST (stress) vs non-stress phase for each subject? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

phases = dr.Phases.PHASE_ORDER


In [3]:
importlib.reload(train)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
    train.Metrics.RESP,
    train.Metrics.MEAN_WRIST_ACT, 
    train.Metrics.PEAK_WRIST_ACC 
]

model_phases = [
    [
        dr.Phases.BASE,
        dr.Phases.FUN,
        dr.Phases.TSST,
        dr.Phases.MEDI_1,
        dr.Phases.MEDI_2
    ],
    [
        dr.Phases.BASE,
        dr.Phases.TSST,
        dr.Phases.MEDI_1,
        dr.Phases.MEDI_2
    ],
    [
        dr.Phases.BASE,
        dr.Phases.MEDI_1,
        dr.Phases.MEDI_2
    ],
    # [dr.Phases.BASE,]
]

label_type = "stai"

for phases in model_phases:
    print(f"PHASES: {phases} " + "-"*30)
    normalize = True
    # normalize = False
    label_type = "all"
    x, y = train.Train_WESAD.get_wesad_data(metrics, phases, verbose=False, label_type=label_type, normalize=normalize)
    # 0-1 scaling
    for i in range(2, len(x.columns)):
        if x.columns[i] in metrics:
            data_col = x[x.columns[i]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x[x.columns[i]] = data_col

    models = {
        "SVM": SVC(C=10, gamma=0.01), 
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "LogReg": LogisticRegression(max_iter=1000),
        "Bayes": GaussianNB(),
        "DT": DecisionTreeClassifier()
    }

    results = {
        "SVM": [], 
        "KNN": [],
        "LogReg": [],
        "Bayes": [],
        "DT": []
    }
    num_iters = 10
    for _ in range(num_iters):
        out = train.train_predict(models, x, y, by_subject=False, test_size=0.12, show_classification=False)
        for model_name in results:
            results[model_name].append(out[model_name])

    for model_name in results.keys():
        print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")

PHASES: ['Base', 'Fun', 'TSST', 'Medi_1', 'Medi_2'] ------------------------------
SVM accuracy over 10 rounds: 0.775
KNN accuracy over 10 rounds: 0.775
LogReg accuracy over 10 rounds: 0.7625
Bayes accuracy over 10 rounds: 0.725
DT accuracy over 10 rounds: 0.7625
PHASES: ['Base', 'TSST', 'Medi_1', 'Medi_2'] ------------------------------
SVM accuracy over 10 rounds: 0.7333333333333333
KNN accuracy over 10 rounds: 0.7333333333333333
LogReg accuracy over 10 rounds: 0.7999999999999999
Bayes accuracy over 10 rounds: 0.8166666666666667
DT accuracy over 10 rounds: 0.8333333333333334
PHASES: ['Base', 'Medi_1', 'Medi_2'] ------------------------------
SVM accuracy over 10 rounds: 0.5599999999999999
KNN accuracy over 10 rounds: 0.6799999999999999
LogReg accuracy over 10 rounds: 0.48
Bayes accuracy over 10 rounds: 0.72
DT accuracy over 10 rounds: 0.6799999999999999
